<a href="https://colab.research.google.com/github/desdesmond/GenAICourse/blob/main/Fine_tune_sentiment_into_emotions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Udacity GenAI Nanodegree**
# Project exercise: Classic fine tuning a pretrained model vs. fine tuning with a loRA adapter


Prepare HF Dataset for fine tuning

In [4]:
# Load HF Dataset

!pip install datasets
from datasets import load_dataset
dataset = load_dataset("dair-ai/emotion")
dataset = dataset["train"].train_test_split(test_size=0.2) # 20% for test, remaining for training
splits = ["train", "test"]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/9.05k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/1.03M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/127k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/129k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/16000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [5]:
# dataset["test"][0]
dataset["test"]["text"][0]

'i feel a little sentimental about because i distinctly remember as a child celebrating my parents th birthdays and they seemed so'

In [5]:
# Get number of labels from the dataset
# atr: https://github.com/achimoraites/machine-learning-playground/blob/main/NLP/Text%20classification/Lightweight_RoBERTa_PEFT_LORA_FineTuning.ipynb
num_labels = dataset['train'].features['label'].num_classes
class_names = dataset["train"].features["label"].names
print(f"number of labels: {num_labels}")
print(f"the labels: {class_names}")

number of labels: 6
the labels: ['sadness', 'joy', 'love', 'anger', 'fear', 'surprise']


In [19]:
# Let's see what the first 10 labels in the Test split

print(f'\n Original Label: {dataset["test"]["label"][0:9]}')


 Original Label: [0, 3, 4, 4, 0, 1, 1, 0, 3]


In [20]:
# I will be using a bert imdb seq classifer. Let's see how this pretrained model work with 10 of the dataset entries
from transformers import pipeline

pipe = pipeline("text-classification", model="ykacer/bert-base-cased-imdb-sequence-classification")

pipe(dataset["test"]["text"][0:9])

[{'label': 'LABEL_1', 'score': 0.6139788627624512},
 {'label': 'LABEL_1', 'score': 0.6027233600616455},
 {'label': 'LABEL_1', 'score': 0.6092396378517151},
 {'label': 'LABEL_1', 'score': 0.6169024705886841},
 {'label': 'LABEL_1', 'score': 0.6188215017318726},
 {'label': 'LABEL_1', 'score': 0.5927574038505554},
 {'label': 'LABEL_1', 'score': 0.5805122256278992},
 {'label': 'LABEL_1', 'score': 0.6319922804832458},
 {'label': 'LABEL_1', 'score': 0.5799717903137207}]

The dataset has 6 labels or outputs, but this checkpoint has only 2. I will now train this checkpoint with this dataset and adjust its output according to the dataset.

In [6]:
# Load a pre-trained bert base model
from transformers import AutoTokenizer, AutoModelForSequenceClassification

checkpoint = "ykacer/bert-base-cased-imdb-sequence-classification"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(
    checkpoint,
    num_labels=num_labels,
    id2label={i: label for i, label in enumerate(class_names)},
    label2id={label: i for i, label in enumerate(class_names)},
    ignore_mismatched_sizes=True # the checkpoint had only 2 output, but now we are training this with 6 output.
    )



The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/284 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/617 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/433M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ykacer/bert-base-cased-imdb-sequence-classification and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([2, 768]) in the checkpoint and torch.Size([6, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([2]) in the checkpoint and torch.Size([6]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
tokenized_dataset = {}
for split in splits:
    tokenized_dataset[split] = dataset[split].map(
        lambda x: tokenizer(x["text"], truncation=True), batched=True
    )

Map:   0%|          | 0/12800 [00:00<?, ? examples/s]

Map:   0%|          | 0/3200 [00:00<?, ? examples/s]

In [30]:
# Let's see what's the dataset looks like.

tokenized_dataset["train"]

Dataset({
    features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 12800
})

In [2]:
# Unfreeze param, for some reason the checkpoint does not let param to be frozen, this block has been skipped.

# for param in model.parameters():
#     param.requires_grad = True
    # print(model)

In [8]:
# Setup evaluation metrics

import numpy as np
import os
from transformers import DataCollatorWithPadding, Trainer, TrainingArguments

# Setup evaluation metrics. Attributed to GenAI course from Udacity
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean()}


## Here, it's a classic training setup.

In [9]:
# Trainer setup. Attributed to GenAI course from Udacity
trainer = Trainer(
    model=model,
    args=TrainingArguments(
        output_dir="./data/trained_with_six_emotions",
        # Set the learning rate
        learning_rate=2e-5,
        # Set the per device train batch size and eval batch size
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        # Evaluate and save the model after each epoch
        evaluation_strategy="epoch",
        save_strategy="epoch",
        # Set the learning rate
        num_train_epochs=5,
        weight_decay=0.01,
        report_to="none",
        load_best_model_at_end=True,
    ),
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-9-e2e250665a96>:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [10]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.659000,0.200204,0.934063
2,0.154700,0.164576,0.932187
3,0.120700,0.211495,0.928125
4,0.083900,0.217664,0.927813
5,0.050500,0.245024,0.928438


TrainOutput(global_step=4000, training_loss=0.1813248553276062, metrics={'train_runtime': 218.4833, 'train_samples_per_second': 292.929, 'train_steps_per_second': 18.308, 'total_flos': 1588047361881984.0, 'train_loss': 0.1813248553276062, 'epoch': 5.0})

In [11]:
# Save and share my checkpoint on HF
from huggingface_hub import notebook_login
notebook_login()


In [12]:
trainer.push_to_hub("desdesmond/emotion-classifer")

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/433M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.30k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/desdesmond/trained_with_six_emotions/commit/809564031850f575046791676eee4de37a897adf', commit_message='desdesmond/emotion-classifer', commit_description='', oid='809564031850f575046791676eee4de37a897adf', pr_url=None, repo_url=RepoUrl('https://huggingface.co/desdesmond/trained_with_six_emotions', endpoint='https://huggingface.co', repo_type='model', repo_id='desdesmond/trained_with_six_emotions'), pr_revision=None, pr_num=None)

In [14]:
# Inference via HF's Pipline method

from transformers import pipeline

pipe = pipeline("text-classification", model="desdesmond/trained_with_six_emotions")

pipe(" I've been waiting for a HuggingFace course my whole life.")

config.json:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/433M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/669k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


[{'label': 'joy', 'score': 0.31162890791893005}]

Compare the original outcome from our checkpoint:

{'label': 'LABEL_1', 'score': 0.6139788627624512}

{'label': 'LABEL_1', 'score': 0.6027233600616455}

{'label': 'LABEL_1', 'score': 0.6092396378517151}

{'label': 'LABEL_1', 'score': 0.6169024705886841}

{'label': 'LABEL_1', 'score': 0.6188215017318726}

{'label': 'LABEL_1', 'score': 0.5927574038505554}

{'label': 'LABEL_1', 'score': 0.5805122256278992}

{'label': 'LABEL_1', 'score': 0.6319922804832458}

{'label': 'LABEL_1', 'score': 0.5799717903137207}

In [16]:
pipe(dataset["test"]["text"][0:9])

[{'label': 'joy', 'score': 0.9987119436264038},
 {'label': 'sadness', 'score': 0.997575581073761},
 {'label': 'joy', 'score': 0.9934242963790894},
 {'label': 'joy', 'score': 0.9879966378211975},
 {'label': 'sadness', 'score': 0.9979508519172668},
 {'label': 'joy', 'score': 0.9989268183708191},
 {'label': 'sadness', 'score': 0.9981295466423035},
 {'label': 'love', 'score': 0.6718432903289795},
 {'label': 'joy', 'score': 0.9986250400543213}]

##Here, I am training the same pre-traied model as before, but with a loRA adapter.

In [17]:
from peft import LoraConfig, get_peft_model

# Let's see if we can improve my trained model with loRA
lora_checkpoint_model = AutoModelForSequenceClassification.from_pretrained(
    checkpoint,
    # "desdesmond/trained_with_six_emotions",
    num_labels=num_labels,
    id2label={i: label for i, label in enumerate(class_names)},
    label2id={label: i for i, label in enumerate(class_names)},
    ignore_mismatched_sizes=True,
    )

print(lora_checkpoint_model)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ykacer/bert-base-cased-imdb-sequence-classification and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([2, 768]) in the checkpoint and torch.Size([6, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([2]) in the checkpoint and torch.Size([6]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [19]:
lora_config = LoraConfig(
    r=4,
    # target_modules=["c_proj"], # This is commented out because I am not sure which module to target on a Bert model
    lora_alpha=8,
    lora_dropout=0.05,
    bias="none",
    task_type="TOKEN_CLS", # experimented with task_type and finally figure out that Token Classification is the correct task
)

lora_model = get_peft_model(lora_checkpoint_model, lora_config)

In [20]:
# Attr. pacman100 hhttps://github.com/huggingface/peft/issues/41#issuecomment-1404611868

def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [21]:
# Let's check how many params are trainable with this lora_config
print_trainable_parameters(lora_checkpoint_model)

trainable params: 152070 || all params: 108466956 || trainable%: 0.14019938016883224


In [22]:
# Setup the loRA trainer

lora_trainer = Trainer(
    model=lora_model,
    args=TrainingArguments(
        output_dir="./data/lora_training",
        learning_rate=2e-5,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        num_train_epochs=5,
        weight_decay=0.01,
        report_to="none",
        # load_best_model_at_end=True, # To-do, need to experiment with this again as now we are using the correct task in arg.
    ),
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),

)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-22-0765eeb2f968>:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  lora_trainer = Trainer(


In [23]:
lora_trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,1.603800,1.508210,0.454688
2,1.305000,1.209629,0.546562
3,1.172700,1.135228,0.577500
4,1.119900,1.106208,0.594688
5,1.095900,1.096914,0.597500


TrainOutput(global_step=4000, training_loss=1.2529230194091796, metrics={'train_runtime': 186.3056, 'train_samples_per_second': 343.522, 'train_steps_per_second': 21.47, 'total_flos': 1590866871070464.0, 'train_loss': 1.2529230194091796, 'epoch': 5.0})

In [24]:
lora_trainer.push_to_hub("desdesmond/lora_training")

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/615k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.24k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/desdesmond/lora_training/commit/4351348c48296ebe8d4f038bcfae1c571fa47ca2', commit_message='desdesmond/lora_training', commit_description='', oid='4351348c48296ebe8d4f038bcfae1c571fa47ca2', pr_url=None, repo_url=RepoUrl('https://huggingface.co/desdesmond/lora_training', endpoint='https://huggingface.co', repo_type='model', repo_id='desdesmond/lora_training'), pr_revision=None, pr_num=None)

In [25]:
lora_trainer.evaluate()

{'eval_loss': 1.0969138145446777,
 'eval_accuracy': 0.5975,
 'eval_runtime': 3.1783,
 'eval_samples_per_second': 1006.815,
 'eval_steps_per_second': 62.926,
 'epoch': 5.0}

In [28]:
# Inferance with my loRA adapter on the original pretrained model

!pip install torch peft

from transformers import AutoModelForCausalLM, AutoModelForSequenceClassification, AutoTokenizer
from peft import PeftModel, PeftConfig
import torch

text = "This is my worst nightmare"
base_model = "ykacer/bert-base-cased-imdb-sequence-classification"
adapter_model = "desdesmond/lora_training"

# Comment out the next block if running the whole notebook
num_labels = 6
id2label = {0: "sadness", 1: "joy", 2: "fear", 3: "anger", 4: "surprise", 5: "love"}
# label2id = {v: k for k, v in id2label.items()}
class_names = list(id2label.values())

model = AutoModelForSequenceClassification.from_pretrained(
    base_model,
    num_labels=num_labels,
    id2label={i: label for i, label in enumerate(class_names)},
    label2id={label: i for i, label in enumerate(class_names)},
    ignore_mismatched_sizes=True,
    )
model.load_adapter(adapter_model)

tokenizer = AutoTokenizer.from_pretrained(base_model)
inputs = tokenizer(text, return_tensors="pt")


with torch.no_grad():
    logits = model(**inputs).logits



Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ykacer/bert-base-cased-imdb-sequence-classification and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([2, 768]) in the checkpoint and torch.Size([6, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([2]) in the checkpoint and torch.Size([6]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


adapter_model.safetensors:   0%|          | 0.00/615k [00:00<?, ?B/s]

In [30]:
print(f'\n Label:{model.config.id2label[predicted_class_id]}, Text:{text}')


 Label:sadness, Text:This is my worst nightmare



**Results from loRA adapter:**
Training Loss
1.603800
1.305000
1.172700
1.119900
1.095900

Validation Loss
1.508210
1.209629
1.135228
1.106208
1.096914

Accuracy
0.454688
0.546562
0.577500
0.594688
0.597500

**Results from classic training:**
Training Loss
0.659000
0.154700
0.120700
0.083900
0.050500

Validation Loss
0.200204
0.164576
0.211495
0.217664
0.245024

Accuracy
0.934063
0.932187
0.928125
0.927813
0.928438

After 5 Epochs on both training, tt seems that the loRA adapter has not been able to achieve the same accuracy as the classic training. Perhaps further experimentations on target layers and randomization of the dataset could help.